In [1]:
pip install opencv-python opencv-python-headless pytube


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python yt-dlp


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install streamlink


Note: you may need to restart the kernel to use updated packages.


In [4]:
# Update this path to where you saved coco.names
coco_names_path = "/Users/chiragn/Desktop/TIMES SQUARE/coco.names"  # Change this to your path

with open(coco_names_path, "r") as f:
    classes = [line.strip() for line in f.readlines()]


In [5]:
pip install pafy youtube_dl opencv-python numpy

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install --upgrade --force-reinstall "git+https://github.com/ytdl-org/youtube-dl.git"

  Cloning https://github.com/ytdl-org/youtube-dl.git to /private/var/folders/47/bh11rgr97kv3n95hy38kv3540000gn/T/pip-req-build-ulx8udlk
  Running command git clone --filter=blob:none --quiet https://github.com/ytdl-org/youtube-dl.git /private/var/folders/47/bh11rgr97kv3n95hy38kv3540000gn/T/pip-req-build-ulx8udlk
  Resolved https://github.com/ytdl-org/youtube-dl.git to commit c5098961b04ce83f4615f2a846c84f803b072639
  Preparing metadata (setup.py) ... done
  Created wheel for youtube-dl: filename=youtube_dl-2021.12.17-py2.py3-none-any.whl size=1969895 sha256=a984df4b53cd8c97674311ea526b518df89372e051126801638929d81353f040
  Stored in directory: /private/var/folders/47/bh11rgr97kv3n95hy38kv3540000gn/T/pip-ephem-wheel-cache-zf4jiwc7/wheels/91/cb/b5/629bb9aeb351f550a453ee0e4cccdea4c8cf4a16074cb0bd4e
Successfully built youtube-dl
  Attempting uninstall: youtube-dl
    Found existing installation: youtube-dl 2021.12.17
    Uninstalling youtube-dl-2021.12.17:
      Successfully uninstalled yo

In [9]:
pip install yt-dlp opencv-python numpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import numpy as np
from datetime import datetime
import yt_dlp
import os

def load_yolo_model(weights_path, cfg_path):
    try:
        net = cv2.dnn.readNet(weights_path, cfg_path)
        layer_names = net.getLayerNames()
        output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
        return net, output_layers
    except Exception as e:
        print(f"Error loading YOLO model: {e}")
        return None, None

def get_youtube_stream(url):
    ydl_opts = {'format': 'best'}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        return info['url']

def log_output(message):
    with open("outputv3.txt", "a") as f:  # Change to outputv3.txt
        f.write(f"{message}\n")
    print(message)

def process_live_video(video_url):
    # Load YOLO model
    base_path = "/Users/chiragn/Desktop/TIMES SQUARE"
    weights_path = os.path.join(base_path, "yolov3.weights")
    cfg_path = os.path.join(base_path, "yolov3.cfg")
    coco_names_path = os.path.join(base_path, "coco.names")

    # Debug: Check if files exist and print their sizes
    log_output(f"weights_path exists: {os.path.exists(weights_path)}, size: {os.path.getsize(weights_path) if os.path.exists(weights_path) else 'N/A'} bytes")
    log_output(f"cfg_path exists: {os.path.exists(cfg_path)}, size: {os.path.getsize(cfg_path) if os.path.exists(cfg_path) else 'N/A'} bytes")
    log_output(f"coco_names_path exists: {os.path.exists(coco_names_path)}, size: {os.path.getsize(coco_names_path) if os.path.exists(coco_names_path) else 'N/A'} bytes")

    net, output_layers = load_yolo_model(weights_path, cfg_path)

    if net is None:
        log_output("Failed to load YOLO model. Exiting.")
        return

    # Load COCO class names
    try:
        with open(coco_names_path, "r") as f:
            classes = [line.strip() for line in f.readlines()]
        log_output(f"Loaded {len(classes)} class names")
        log_output("First 10 class names: " + str(classes[:10]))  # Print first 10 class names for debugging
    except Exception as e:
        log_output(f"Error loading class names: {e}")
        # Try to print file contents
        try:
            with open(coco_names_path, "r") as f:
                log_output("File contents:")
                log_output(f.read())
        except Exception as e:
            log_output(f"Error reading coco.names file: {e}")
        return

    # Get the actual stream URL
    try:
        stream_url = get_youtube_stream(video_url)
        log_output(f"Successfully retrieved stream URL: {stream_url[:50]}...")  # Print first 50 chars of URL
    except Exception as e:
        log_output(f"Error getting stream URL: {e}")
        return

    cap = cv2.VideoCapture(stream_url)

    if not cap.isOpened():
        log_output("Error opening video stream or file")
        return

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            log_output("Failed to capture frame. Exiting.")
            break

        frame_count += 1
        if frame_count % 30 == 0:  # Process every 30th frame
            # Perform YOLO detection on the frame
            blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
            net.setInput(blob)
            outs = net.forward(output_layers)

            # Process the outputs to extract bounding boxes, confidences, etc.
            class_ids = []
            confidences = []
            boxes = []
            height, width, _ = frame.shape

            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5:  # Confidence threshold
                        center_x = int(detection[0] * width)
                        center_y = int(detection[1] * height)
                        w = int(detection[2] * width)
                        h = int(detection[3] * height)

                        # Rectangle coordinates
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)

                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

            # Apply Non-Maxima Suppression
            indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

            # Count heads detected (assuming class_id for heads is known)
            head_count = 0
            for i in range(len(boxes)):
                if i in indexes:
                    try:
                        x, y, w, h = boxes[i]
                        class_id = class_ids[i]
                        if 0 <= class_id < len(classes):
                            label = str(classes[class_id])
                            # Assuming the label for heads is 'person'
                            if label == 'person':
                                head_count += 1
                                color = (0, 255, 0)  # Green color
                                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                                cv2.putText(frame, label, (x, y + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                        else:
                            log_output(f"Warning: class_id {class_id} is out of range")
                    except Exception as e:
                        log_output(f"Error processing detection: {e}")

            # Log head count with timestamp
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            log_output(f"[{current_time}] Head Count: {head_count}")

            # Show the frame
            cv2.imshow("Live Video", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Replace with your live video URL
    live_video_url = "https://www.youtube.com/live/PQQ8EPRFLzo?si=vNT8GhTT89859XA8"
    process_live_video(live_video_url)


weights_path exists: True, size: 248007048 bytes
cfg_path exists: True, size: 8342 bytes
coco_names_path exists: True, size: 624 bytes
Loaded 80 class names
First 10 class names: ['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light']
[youtube] Extracting URL: https://www.youtube.com/live/PQQ8EPRFLzo?si=vNT8GhTT89859XA8
[youtube] PQQ8EPRFLzo: Downloading webpage
[youtube] PQQ8EPRFLzo: Downloading ios player API JSON
[youtube] PQQ8EPRFLzo: Downloading web creator player API JSON
[youtube] PQQ8EPRFLzo: Downloading m3u8 information
[youtube] PQQ8EPRFLzo: Downloading m3u8 information
Successfully retrieved stream URL: https://manifest.googlevideo.com/api/manifest/hls_...
[2024-10-01 10:41:31] Head Count: 23
[2024-10-01 10:41:32] Head Count: 22
[2024-10-01 10:41:33] Head Count: 16
[2024-10-01 10:41:33] Head Count: 19


2024-10-01 10:41:33.334 python[20752:444593] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-01 10:41:33.334 python[20752:444593] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


[2024-10-01 10:41:33] Head Count: 16
[2024-10-01 10:41:34] Head Count: 18
[2024-10-01 10:41:34] Head Count: 18
[2024-10-01 10:41:34] Head Count: 14
[2024-10-01 10:41:34] Head Count: 14
[2024-10-01 10:41:35] Head Count: 21
[2024-10-01 10:41:36] Head Count: 18
[2024-10-01 10:41:37] Head Count: 22
[2024-10-01 10:41:37] Head Count: 23
[2024-10-01 10:41:37] Head Count: 20
[2024-10-01 10:41:39] Head Count: 22
[2024-10-01 10:41:39] Head Count: 30
[2024-10-01 10:41:39] Head Count: 22
[2024-10-01 10:41:40] Head Count: 23
[2024-10-01 10:41:40] Head Count: 24
[2024-10-01 10:41:45] Head Count: 23
[2024-10-01 10:41:46] Head Count: 18
[2024-10-01 10:41:46] Head Count: 20
[2024-10-01 10:41:46] Head Count: 20
[2024-10-01 10:41:46] Head Count: 19
[2024-10-01 10:41:47] Head Count: 24
[2024-10-01 10:41:47] Head Count: 22
[2024-10-01 10:41:47] Head Count: 21
[2024-10-01 10:41:47] Head Count: 20
[2024-10-01 10:41:48] Head Count: 25
[2024-10-01 10:41:51] Head Count: 21
[2024-10-01 10:41:51] Head Count: 28
[

KeyboardInterrupt: 

: 

In [3]:
import os

weights_path = "/Users/chiragn/Desktop/TIMES SQUARE/yolov3.weights"  # Update with your path
cfg_path = "/Users/chiragn/Desktop/TIMES SQUARE/yolov3.cfg"          # Update with your path

# Check if files exist
if not os.path.isfile(weights_path):
    print(f"Error: Weights file does not exist at {weights_path}")
else:
    print(f"Weights file exists at {weights_path}")

if not os.path.isfile(cfg_path):
    print(f"Error: Config file does not exist at {cfg_path}")
else:
    print(f"Config file exists at {cfg_path}")


Weights file exists at /Users/chiragn/Desktop/TIMES SQUARE/yolov3.weights
Config file exists at /Users/chiragn/Desktop/TIMES SQUARE/yolov3.cfg
